# Querying Philadelphia Phillies with LlamaIndex and NebulaGraph

## Installation and Configuration

In [3]:
%pip install -U llama_index ipython-ngql nebula3-python pyvis networkx youtube_transcript_api 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
     ---------------------------------------- 0.1/15.4 MB 3.2 MB/s eta 0:00:05
     --------------------------------------- 0.1/15.4 MB 787.7 kB/s eta 0:00:20
     ---------------------------------------- 0.1/15.4 MB 1.0 MB/s eta 0:00:15
     ---------------------------------------- 0.1/15.4 MB 1.0 MB/s eta 0:00:15
     --------------------------------------- 0.2/15.4 MB 841.6 kB/s eta 0:00:19
     --------------------------------------- 0.2/15.4 MB 841.6 kB/s eta 0:00:19
      -------------------------------------- 0.2/15.4 MB 758.5 kB/s eta 0:00:21
      -------------------------------------- 0.3/15.4 MB 827.5 kB/s eta 0:00:19
      -------------------------------------- 0.3/15.4 MB 759.5 kB/s eta 0:00:20
      -------------------------------------- 0.3/15.4 MB 756.6 kB/s eta 0:00:20
      -------------------------------------- 0.4/15.4 MB 740.5 kB/s et

In [2]:
import os
import logging
import sys

os.environ["OPENAI_API_KEY"] = "sk-#################"

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

## Connect to Nebula Graph and set up a new space

In [3]:
os.environ["GRAPHD_HOST"] = "127.0.0.1"
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula" 
os.environ["NEBULA_ADDRESS"] = "127.0.0.1:9669"  

%reload_ext ngql
connection_string = f"--address {os.environ['GRAPHD_HOST']} --port 9669 --user root --password {os.environ['NEBULA_PASSWORD']}"
%ngql {connection_string}

Connection Pool Created


,Name
0,demo_basketballplayer
1,phillies_rag


In [4]:
%ngql CREATE SPACE IF NOT EXISTS phillies_rag(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

""


In [5]:
%ngql SHOW SPACES;

,Name
0,demo_basketballplayer
1,phillies_rag


In [6]:
%%ngql
USE phillies_rag;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);

""


In [7]:
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

""


In [4]:
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

space_name = "phillies_rag"
edge_types, rel_prop_names = ["relationship"], ["relationship"]
tags = ["entity"]

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

ModuleNotFoundError: No module named 'llama_index.storage'

## Load data and create KG index

In [9]:
from llama_index import (
    LLMPredictor,
    ServiceContext,
    KnowledgeGraphIndex,
)
from llama_index.graph_stores import SimpleGraphStore
from llama_index import download_loader
from llama_index.llms import OpenAI

# define LLM
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

In [10]:
from llama_index import load_index_from_storage
from llama_hub.youtube_transcript import YoutubeTranscriptReader

try:

    storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
    kg_index = load_index_from_storage(
        storage_context=storage_context,
        service_context=service_context,
        max_triplets_per_chunk=15,
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
        verbose=True,
    )
    index_loaded = True
except:
    index_loaded = False

if not index_loaded:
    
    WikipediaReader = download_loader("WikipediaReader")
    loader = WikipediaReader()
    wiki_documents = loader.load_data(pages=['Philadelphia Phillies'], auto_suggest=False)
    print(f'Loaded {len(wiki_documents)} documents')

    youtube_loader = YoutubeTranscriptReader()
    youtube_documents = youtube_loader.load_data(ytlinks=['https://www.youtube.com/watch?v=k-HTQ8T7oVw'])    
    print(f'Loaded {len(youtube_documents)} YouTube documents')

    kg_index = KnowledgeGraphIndex.from_documents(
        documents=wiki_documents + youtube_documents,
        storage_context=storage_context,
        max_triplets_per_chunk=15,
        service_context=service_context,
        space_name=space_name,
        edge_types=edge_types,
        rel_prop_names=rel_prop_names,
        tags=tags,
        include_embeddings=True,
    )
    
    kg_index.storage_context.persist(persist_dir='./storage_graph')

## Query with Text2Cypher

In [11]:
from llama_index.query_engine import KnowledgeGraphQueryEngine
from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from IPython.display import Markdown, display

# query_engine = KnowledgeGraphQueryEngine(
#     storage_context=storage_context,
#     service_context=service_context,
#     llm=llm,
#     verbose=True,
# )

hybrid_query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=3,
    explore_global_knowledge=True,
)

In [12]:
hybrid_query_engine = kg_index.as_query_engine()
response = hybrid_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a baseball player who was named MVP of the NLCS. He also fueled tensions after signing a 13-year $330 million contract.</b>

In [13]:
query_engine = kg_index.as_query_engine()
response = query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a baseball player who was named MVP of the NLCS. He also fueled tensions after signing a 13-year $330 million contract.</b>

In [14]:
response = query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a baseball player who was named MVP of the NLCS. He also fueled tensions after signing a 13-year $330 million contract.</b>

In [15]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Bryce Harper'
RETURN p, e, m;

,p,e,m
0,"(""Bryce Harper"" :entity{name: ""Bryce Harper""})","(""Bryce Harper"")-[:relationship@45106556397841...","(""13-year $330 million contract"" :entity{name:..."
1,"(""Bryce Harper"" :entity{name: ""Bryce Harper""})","(""Bryce Harper"")-[:relationship@59824214907797...","(""NLCS"" :entity{name: ""NLCS""})"


In [16]:
%ng_draw

<class 'pyvis.network.Network'> |N|=3 |E|=2

In [17]:
response = query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The impact of the standing ovation Trey Turner received had a positive effect on his season. Following the standing ovation, Trey Turner appeared to be in higher spirits, more at ease, and less burdened by his own thoughts. This translated into improved performance on the field, as evidenced by an increase in his batting average, more successful hits, and a greater number of home runs. The support and admiration displayed by the Philadelphia fans appeared to have bolstered his self-assurance and aided him in overcoming his previous struggles.</b>

In [18]:
%%ngql 
MATCH (p:`entity`)-[r:`relationship`]->(q:`entity`)
WHERE p.`entity`.`name` == 'Trey Turner' 
RETURN p, r, q;

,p,r,q
0,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-89965735714464...","(""as quick"" :entity{name: ""as quick""})"
1,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-89414636367958...","(""a standing ovation"" :entity{name: ""a standin..."
2,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-89275189104623...","(""in comments"" :entity{name: ""in comments""})"
3,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-86268653017792...","(""25 hits"" :entity{name: ""25 hits""})"
4,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@-86268653017792...","(""5 home runs"" :entity{name: ""5 home runs""})"
...,...,...,...
70,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@804341681940123...","(""ball and strike call"" :entity{name: ""ball an..."
71,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@859994387799338...","(""baseball"" :entity{name: ""baseball""})"
72,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@859994387799338...","(""fifth home run"" :entity{name: ""fifth home ru..."
73,"(""Trey Turner"" :entity{name: ""Trey Turner""})","(""Trey Turner"")-[:relationship@878913129174356...","(""inside of his own head"" :entity{name: ""insid..."


In [19]:
%ng_draw

<class 'pyvis.network.Network'> |N|=75 |E|=75

In [20]:
response = query_engine.query("Tell me about some of the facts of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The Philadelphia Phillies are a professional baseball team based in Philadelphia. They compete in Major League Baseball as a member of the National League East division. The team's home stadium is Citizens Bank Park, located in the South Philadelphia Sports Complex. The Phillies have a rich history, with two World Series championships (in 1980 and 2008), eight National League pennants, and 15 playoff appearances. They have played over 21,000 games since their establishment in 1883, with a record of 10,022 wins and 11,187 losses. The Phillies have had successful periods, including winning 11 division titles and making five consecutive playoff appearances from 2007 to 2011. They have also had notable players, such as Hall of Famers Richie Ashburn and Robin Roberts, as well as recent stars like Bryce Harper, who won the NL Most Valuable Player Award in 2021.</b>

In [21]:
%%ngql 
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Philadelphia Phillies' 
RETURN p, e, m;

,p,e,m
0,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@-9215...","(""Joe Girardi"" :entity{name: ""Joe Girardi""})"
1,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@-8815...","(""Major League Baseball"" :entity{name: ""Major ..."
2,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@-4023...","(""15 playoff appearances"" :entity{name: ""15 pl..."
3,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@14545...","(""National League"" :entity{name: ""National Lea..."
4,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@26691...","(""American professional baseball team"" :entity..."
5,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@30666...","(""Lehigh Valley IronPigs"" :entity{name: ""Lehig..."
6,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@42124...","(""Philadelphia"" :entity{name: ""Philadelphia""})"
7,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@91846...","(""eight National League pennants"" :entity{name..."
8,"(""Philadelphia Phillies"" :entity{name: ""Philad...","(""Philadelphia Phillies"")-[:relationship@91846...","(""two World Series championships"" :entity{name..."


In [22]:
%ng_draw

<class 'pyvis.network.Network'> |N|=10 |E|=9

## Query with vector index

In [23]:
from llama_index import VectorStoreIndex

WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
wiki_documents = loader.load_data(pages=['Philadelphia Phillies'], auto_suggest=False)
print(f'Loaded {len(wiki_documents)} documents')

youtube_loader = YoutubeTranscriptReader()
youtube_documents = youtube_loader.load_data(ytlinks=['https://www.youtube.com/watch?v=k-HTQ8T7oVw'])    
print(f'Loaded {len(youtube_documents)} YouTube documents')

vector_index = VectorStoreIndex.from_documents(wiki_documents + youtube_documents)
vector_query_engine = vector_index.as_query_engine()

response = vector_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

Loaded 1 documents
Loaded 1 YouTube documents


<b>Bryce Harper is a professional baseball player who has played for the Philadelphia Phillies since the 2019 season. He was signed to a 13-year, $330 million deal, making it one of the biggest contracts in baseball history. Harper has been a key player for the Phillies and has had notable achievements, including winning the NL Most Valuable Player Award for the 2021 season. He has been a significant contributor to the team's success, although they have not yet made the playoffs during his tenure.</b>

In [24]:
response = vector_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received seemed to have a positive impact on his season. After receiving the ovation, Turner had one of his best games in weeks and felt that the hit he got during that game was one of his best swings all year. In the 18 games since the standing ovation, Turner has been performing well, with a batting average of .343, an OPS over a thousand, and several hits, home runs, and RBIs. The love and support shown by the city of Philadelphia seemed to have boosted Turner's confidence and helped him relax, resulting in improved performance on the field.</b>

In [25]:
response = vector_query_engine.query("Tell me about some of the facts of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The Philadelphia Phillies are a professional baseball team based in Philadelphia. They compete in Major League Baseball as a member of the National League East division. The team's home stadium is Citizens Bank Park, located in the South Philadelphia Sports Complex. The Phillies have a history dating back to 1883 and have played at several stadiums in the city over the years. They have won two World Series championships, eight National League pennants, and have made 15 playoff appearances. The team has a loyal fan base and is known for its longevity. The Phillies have had both periods of success and periods of futility throughout their history. They were the last of the pre-expansion teams to win a World Series, with their first championship coming in 1980. Since the start of the Divisional Era in 1969, the Phillies have been one of the most successful teams in MLB, winning 11 division titles, eight National League pennants, and two World Series championships. The team's Triple-A affiliate is the Lehigh Valley IronPigs, and their Double-A affiliate is the Reading Fightin Phils. The Phillies' spring training facilities are located in Clearwater, Florida.</b>

## Create CustomRetriever to combine vector index and KG index

In [26]:
from llama_index import QueryBundle
from llama_index.schema import NodeWithScore
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever
from typing import List

class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [27]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever, KGTableRetriever

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

## Create 7 query engines and run queries

In [28]:
# KG vector-based entity retrieval
kg_query_engine = kg_index.as_query_engine()

# KG keyword-based entity retrieval
kg_keyword_query_engine = kg_index.as_query_engine(
    # setting to false uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

# KG hybrid entity retrieval
kg_hybrid_query_engine = kg_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=3,
    explore_global_knowledge=True,
)

# Raw vector index retrieval
vector_query_engine = vector_index.as_query_engine()

# Custom combo query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

# using KnowledgeGraphQueryEngine
from llama_index.query_engine import KnowledgeGraphQueryEngine

kgqe_query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

# using KnowledgeGraphRAGRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

kg_rag_query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, service_context=service_context
)

In [29]:
response = kg_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a baseball player who was named MVP of the NLCS. He also fueled tensions after signing a 13-year $330 million contract.</b>

In [30]:
response = kg_keyword_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a person who has been involved in various relationships and events. He was named MVP of the NLCS and also fueled tensions after signing a 13-year $330 million contract.</b>

In [31]:
response = kg_hybrid_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a star outfielder who has made a significant impact in the world of baseball. He is known for his impressive hitting skills and has been a key player in the lineup of the Philadelphia Phillies. Harper signed a lucrative 13-year, $330 million contract with the Phillies as a free agent in 2019, which has generated some tensions. He was named the MVP of the NLCS and also won the NL Most Valuable Player Award in 2021. Harper's contributions have helped the Phillies reach the playoffs in 2022 and advance to the World Series.</b>

In [32]:
response = vector_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a professional baseball player who has played for the Philadelphia Phillies since the 2019 season. He was signed to a 13-year, $330 million deal, making it one of the biggest contracts in baseball history. Harper has been a key player for the Phillies and has had notable achievements, including winning the NL Most Valuable Player Award for the 2021 season. He has been a significant contributor to the team's success, helping them reach the playoffs in the 2022 season and advancing to the World Series.</b>

In [33]:
response = custom_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

<b>Bryce Harper is a professional baseball player who has played for the Philadelphia Phillies. He joined the team in 2019 after signing a 13-year, $330 million contract, which was a significant move as he was taken away from the division rival Washington Nationals. Harper has been a key player for the Phillies and has had notable achievements during his time with the team. In 2021, he won the NL Most Valuable Player Award. Additionally, Harper played a crucial role in the Phillies' successful 2022 season, where they reached the playoffs for the first time since 2011 and advanced to the World Series. He was named the MVP of the NLCS during that postseason. Overall, Bryce Harper is a highly regarded player who has made a significant impact on the Philadelphia Phillies.</b>

In [34]:
response = kgqe_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

Graph Store Query:
MATCH (p:`entity`)-[r:`relationship`]->(e:`entity`)
WHERE p.`entity`.`name` == 'Bryce Harper'
RETURN e.`entity`.`name`
Graph Store Response:
{'e.entity.name': ['13-year $330 million contract', 'NLCS']}
Final Response: Bryce Harper is associated with a 13-year $330 million contract and the NLCS.


<b>Bryce Harper is associated with a 13-year $330 million contract and the NLCS.</b>

In [35]:
response = kg_rag_query_engine.query("Tell me about Bryce Harper.")
display(Markdown(f"<b>{response}</b>"))

Entities processed: ['Bryce', 'Harper', 'Bryce Harper']
Entities processed: ['bryce harper', 'bryce', 'harper']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
Bryce Harper{name: Bryce Harper} -[relationship:{relationship: was named MVP of}]-> NLCS{name: NLCS}
Bryce Harper{name: Bryce Harper} -[relationship:{relationship: fueled tensions after signing}]-> 13-year $330 million contract{name: 13-year $330 million contract}


<b>Bryce Harper is a person who has been involved in various relationships and events.</b>

In [36]:
response = kg_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The impact of the standing ovation Trey Turner received seemed to have a positive effect on his season. After the ovation, Trey appeared to be in higher spirits and more at ease. This change in mindset translated to his performance on the field, as he played the game with increased confidence. In the games that followed the ovation, Trey displayed impressive stats, including a batting average of .343, five home runs, and 15 RBIs. The support and encouragement from the Philadelphia fans appeared to have revitalized Trey and contributed to his success during that period.</b>

In [37]:
response = kg_keyword_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received likely had a positive impact on his season.</b>

In [38]:
response = kg_hybrid_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received had a positive impact on his season. After receiving the ovation, Turner seemed happier, more relaxed, and out of his own head. He started performing better in the games, with improved batting average, more hits, and more home runs. The love and support shown by the fans seemed to have boosted his confidence and helped him get out of his slump. Overall, the standing ovation played a significant role in changing Trey Turner's season for the better.</b>

In [39]:
response = vector_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received seemed to have a positive impact on his season. After receiving the ovation, Turner had one of his best games in weeks and felt that the hit he got during that game was one of his best swings all year. In the 18 games since the standing ovation, Turner has been performing significantly better, with a batting average of .343, an OPS over a thousand, and multiple hits, home runs, and RBIs. The love and support shown by the Philadelphia fans seemed to have boosted Turner's confidence and helped him relax, resulting in improved performance on the field.</b>

In [40]:
response = custom_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

<b>The standing ovation Trey Turner received had a positive impact on his season. After the ovation, Turner seemed happier, more relaxed, and out of his own head. He started performing better in games, having one of his best games in weeks. He mentioned that the hit he got during that game felt like one of his best swings all year. Since the standing ovation, Turner has been batting with a higher average, hitting more home runs, and driving in more runs. The love and support shown by the city of Philadelphia seemed to have boosted his confidence and helped him get out of his slump. Overall, it appears that the standing ovation boosted Turner's confidence and motivation, leading to a positive change in his season.</b>

In [41]:
response = kgqe_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

Graph Store Query:
MATCH (p:`entity`)-[:relationship]->(e:`entity`)
WHERE p.`entity`.`name` == 'Trey Turner' AND e.`entity`.`name` == 'season'
SET p.`entity`.`standing_ovation` = 'changed'
RETURN p.`entity`.`standing_ovation`, e.`entity`.`name`
ERROR:llama_index.graph_stores.nebulagraph:Query failed. Query: MATCH (p:`entity`)-[:relationship]->(e:`entity`)
WHERE p.`entity`.`name` == 'Trey Turner' AND e.`entity`.`name` == 'season'
SET p.`entity`.`standing_ovation` = 'changed'
RETURN p.`entity`.`standing_ovation`, e.`entity`.`name`, Param: {}Error message: Query failed. Query: MATCH (p:`entity`)-[:relationship]->(e:`entity`)
WHERE p.`entity`.`name` == 'Trey Turner' AND e.`entity`.`name` == 'season'
SET p.`entity`.`standing_ovation` = 'changed'
RETURN p.`entity`.`standing_ovation`, e.`entity`.`name`, Param: {}Error message: SyntaxError: syntax error near `MAT'
ERROR:llama_index.graph_stores.nebulagraph:Query failed. Query: MATCH (p:`entity`)-[:relationship]->(e:`entity`)
WHERE p.`entity`.`

RetryError: RetryError[<Future at 0x27094db1990 state=finished raised ValueError>]

In [42]:
response = kg_rag_query_engine.query("How did the standing ovation Trey Turner received change his season?")
display(Markdown(f"<b>{response}</b>"))

Entities processed: ['standing', 'Trey Turner', 'Trey', 'ovation', 'season', 'change', 'standing ovation', 'Turner']
Entities processed: ['standing', 'Trey Turner', 'ovation', 'Trey', 'season', 'change', 'standing ovation', 'Turner']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
standing ovation{name: standing ovation} <-[relationship:{relationship: look back and say}]- Trey Turner{name: Trey Turner} -[relationship:{relationship: has been stuck}]-> inside of his own head{name: inside of his own head}
standing ovation{name: standing ovation} <-[relationship:{relationship: look back and say}]- Trey Turner{name: Trey Turner} -[relationship:{relationship: has been fighting}]-> his own demons{name: his own demons}
standing ovation{name: standing ovation} <-[relationship:{relationship: look back and say}]- Trey

<b>The standing ovation Trey Turner received likely had a positive impact on his season.</b>

In [43]:
response = kg_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is Citizens Bank Park. It opened in 2004 and is located in South Philadelphia. The stadium is part of the South Philadelphia Sports Complex, which also includes the Wells Fargo Center and Lincoln Financial Field. Citizens Bank Park replaced Veterans Stadium as the home of the Phillies.</b>

In [44]:
response = kg_keyword_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is known for its unique feature of featuring cardboard figures.</b>

In [45]:
response = kg_hybrid_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is Citizens Bank Park. It has been the team's home stadium since 2004 and is located in the South Philadelphia Sports Complex.</b>

In [46]:
response = vector_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is Citizens Bank Park, which has been their home since 2004. It is located in the South Philadelphia Sports Complex.</b>

In [47]:
response = custom_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

<b>The current stadium of the Philadelphia Phillies is Citizens Bank Park. It has been the team's home stadium since 2004 and is located in the South Philadelphia Sports Complex. Citizens Bank Park has a restaurant built into the base of the main scoreboard named "Harry the K's" in honor of the late Phillies broadcaster Harry Kalas. The stadium also features a video system that was upgraded in 2011, including a new display screen in left field, which is the largest in the National League at 76 feet high and 97 feet wide.</b>

In [48]:
response = kgqe_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

Graph Store Query:
MATCH (s:`entity`)-[:`relationship`]->(p:`entity`)
WHERE s.`name` == 'Philadelphia Phillies'
RETURN p.`name`
Graph Store Response:
{'p.name': []}
Final Response: I'm sorry, but I couldn't find any facts about the current stadium of the Philadelphia Phillies based on the given context information.


<b>I'm sorry, but I couldn't find any facts about the current stadium of the Philadelphia Phillies based on the given context information.</b>

In [50]:
response = kg_rag_query_engine.query("Tell me some facts about the current stadium of Philadelphia Phillies.")
display(Markdown(f"<b>{response}</b>"))

Entities processed: ['Philadelphia', 'stadium', 'current stadium', 'facts', 'current', 'Philadelphia Phillies', 'Phillies']
Entities processed: ['phillies', 'current stadium', 'Facts', 'facts', 'Phillies', 'philadelphia', 'stadium', 'Philadelphia', 'Stadium', 'current', 'Philadelphia Phillies', 'Current', 'Current stadium']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
Philadelphia Phillies{name: Philadelphia Phillies} <-[relationship:{relationship: rivalry with}]- Washington Nationals{name: Washington Nationals} -[relationship:{relationship: extends back during}]-> original tenure{name: original tenure}
Philadelphia Phillies{name: Philadelphia Phillies} <-[relationship:{relationship: rivalry with}]- Washington Nationals{name: Washington Nationals} -[relationship:{relationship: increased in geographic ten

<b>The current stadium of the Philadelphia Phillies is not mentioned in the given context information.</b>